## Score the Organizations Using the Name, Alternative Name, and Previous Name Fields

In [1]:
### import the libraries used to process the PatentsView and OC data.
import pandas as pd
import numpy as np
import time
import os
import re
import string
import warnings
warnings.filterwarnings('ignore')

### start timer
t0=time.time()

### set the path for the input file and save to variable
res_folder = "../csvResults/"
input_file = "OcResults1000PreparedForScoring.csv"
a_full=os.path.join(res_folder,input_file)

OCScoringDF=pd.read_csv(a_full)

### end timer and print total time
t1 = time.time()
total = t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset 
display(OCScoringDF.info(null_counts=True),OCScoringDF.head())

Total time is 0.000000 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  292 non-null    int64  
 1   assignee_id         292 non-null    object 
 2   location_id         292 non-null    object 
 3   organization        292 non-null    object 
 4   city                292 non-null    object 
 5   state               292 non-null    object 
 6   city_latitude       284 non-null    float64
 7   city_longitude      284 non-null    float64
 8   dateOfFirstPat      292 non-null    object 
 9   nameScores          292 non-null    float64
 10  matchNames          292 non-null    object 
 11  subJurisCode        292 non-null    object 
 12  incorporation_date  292 non-null    object 
 13  address_city        88 non-null     object 
 14  address_state       87 non-null     object 
 15  address_latitude    81 non-nu

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,agent_latitude,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,37.2738,-79.9602,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70


## Scoring Algorithm

In [2]:
totalScore=[]
d=len(OCScoringDF)
x=''

### score the fuzzy match percentages

for s in range(d):
    ### scores all records with a fuzzy match score with a 100%; account for organization
    ### name lengths (i.e., shorter names are scored lower than longer names)
    
    if OCScoringDF['nameScores'][s] == 100:

        if len(OCScoringDF['organization'][s]) < 5:
            x=0
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=2
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=4
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=5
    
    ### visual inspection of the data indicates a discrete group between 95% and 100%,
    ### resulting in the next set. Name lengths are again accounted for and shorter
    ### names/scores are downweighted even more
    
    elif 95 <= OCScoringDF['nameScores'][s] < 100:
        
        if len(OCScoringDF['organization'][s]) < 5:
            x=0
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=1
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=2
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=3

    ### many of the fuzzy matches in this range are wrong, but there are a couple correct
    ### that should not be discounted. While the weights are not as high as the previous
    ### sections, correct matches will be given better scores than the next section
    
    elif 90 <= OCScoringDF['nameScores'][s] < 95:
        
        if len(OCScoringDF['organization'][s]) < 5:
            x=-1
        
        elif 5 <= len(OCScoringDF['organization'][s]) < 10:
            x=1
            
        elif 10 <= len(OCScoringDF['organization'][s]) < 15:
            x=2
            
        elif len(OCScoringDF['organization'][s]) >= 15:
            x=3

    ### scoring the different features that contain state information for each
    ### organization. The jurisdictionScore feature was created by extracting
    ### the state from the jurisdiction_code field. The jurisdiction_code
    ### feature is the primary metric used to match patentsview and OC records
    ### and therefore, given a larger weight. The stateAddScore is given the
    ### second highest weight because it is the primary address that is listed
    ### in an OC record. stateAgtScore is given the least amount of weight
    ### because the agent may not always be located at the registered address 
    ### for the organization.
    
    if OCScoringDF.iloc[s,5] == OCScoringDF.iloc[s,11]:
        x=x+5
        
    elif OCScoringDF.iloc[s,11] == 'De':
        x=x+3
    
    
    if (OCScoringDF.iloc[s,5] == OCScoringDF.iloc[s,14] or OCScoringDF.iloc[s,5] == OCScoringDF.iloc[s,18] or 
        OCScoringDF.iloc[s,5] == OCScoringDF.iloc[s,22]):
        x=x+5
        
    else:
        x=x+0

    
    if (OCScoringDF.iloc[s,4] == OCScoringDF.iloc[s,13] or OCScoringDF.iloc[s,4] == OCScoringDF.iloc[s,17] or 
        OCScoringDF.iloc[s,4] == OCScoringDF.iloc[s,21]):
        x=x+5
        
    else:
        x=x+0
        
    ### the address_city feature is weighted more than the agent_city column for similar
    ### reasons stated in the states section above. Cities less than 4 characters long
    ### are penalized and gradually score better as the character length increase. Moreover,
    ### the score from fuzzy matching is used to create groups as shown below. Fuzzy
    ### scores below 90% are weighted negatively
    

    if (OCScoringDF.iloc[s,25] == 0 or OCScoringDF.iloc[s,26] == 0 or OCScoringDF.iloc[s,27] == 0):
        x=x+5
        
    elif (0 < OCScoringDF.iloc[s,25] < 10 or 0 < OCScoringDF.iloc[s,26] < 10 or 0 < OCScoringDF.iloc[s,27] < 10):
        x=x+4
    
    elif (10 <= OCScoringDF.iloc[s,25] < 50 or 10 <= OCScoringDF.iloc[s,26] < 50 or 10 <= OCScoringDF.iloc[s,27] < 50):
        x=x+3
        
    elif (50 <= OCScoringDF.iloc[s,25] < 100 or 50 <= OCScoringDF.iloc[s,26] < 100 or 50 <= OCScoringDF.iloc[s,27] < 100):
        x=x+2
        
    elif (100 <= OCScoringDF.iloc[s,25] < 200 or 100 <= OCScoringDF.iloc[s,26] < 200 or 100 <= OCScoringDF.iloc[s,27] < 200):
        x=x+1
    
    elif (OCScoringDF.iloc[s,25] >= 200 or OCScoringDF.iloc[s,26] >= 200 or OCScoringDF.iloc[s,27] >= 200):
        x=x-2
    
#     elif (np.isnan(OCScoringDF.iloc[s,27])):
#         x=x+0
            
    
    try:
        if df4['dateDiff'][s] <= 5:
            x=x+5

        elif 5 < df4['dateDiff'][s] <= 10:
            x=x+4
        
        elif 10 < df4['dateDiff'][s] <= 15:
            x=x+3
            
        elif 15 < df4['dateDiff'][s] <= 20:
            x=x+2
            
        elif df4['dateDiff'][s] > 20:
            x=x+1

    except:
        pass
    
    totalScore.append(x)

In [3]:
OCScoringDF['totalScore'] = totalScore

display(OCScoringDF.info(),OCScoringDF.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  292 non-null    int64  
 1   assignee_id         292 non-null    object 
 2   location_id         292 non-null    object 
 3   organization        292 non-null    object 
 4   city                292 non-null    object 
 5   state               292 non-null    object 
 6   city_latitude       284 non-null    float64
 7   city_longitude      284 non-null    float64
 8   dateOfFirstPat      292 non-null    object 
 9   nameScores          292 non-null    float64
 10  matchNames          292 non-null    object 
 11  subJurisCode        292 non-null    object 
 12  incorporation_date  292 non-null    object 
 13  address_city        88 non-null     object 
 14  address_state       87 non-null     object 
 15  address_latitude    81 non-null     float64
 16  address_

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,-79.9602,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11,25
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78,7
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41,5
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56,5
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70,7


In [4]:
OCScoringDF1=OCScoringDF.groupby(by=['assignee_id','organization'],as_index=False)['totalScore'].max()

display(OCScoringDF1.info(),OCScoringDF1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   assignee_id   212 non-null    object
 1   organization  212 non-null    object
 2   totalScore    212 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


None

,assignee_id,organization,totalScore
0,00059b74-f414-43cb-a36e-e30fb41c9285,Qbe Holdings,24
1,004053ef-9a48-4d23-9fd9-93ff47b7c299,Cora Aero,5
2,0116426f-e05d-4c31-bed1-5e811c29400a,Dohrmann Industries,15
3,01756cf3-dcd4-4332-a3ff-a240e090c08f,Ls Biopath,24
4,01a0afe9-4372-4deb-abd5-678d884ed4bb,Sonifi Solutions,16


In [5]:
OCScoringDF1Merge=OCScoringDF1.merge(OCScoringDF,on=['assignee_id','organization','totalScore'],how='inner')

display(OCScoringDF1Merge.info(),OCScoringDF1Merge.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 240
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   assignee_id         241 non-null    object 
 1   organization        241 non-null    object 
 2   totalScore          241 non-null    int64  
 3   ID                  241 non-null    int64  
 4   location_id         241 non-null    object 
 5   city                241 non-null    object 
 6   state               241 non-null    object 
 7   city_latitude       238 non-null    float64
 8   city_longitude      238 non-null    float64
 9   dateOfFirstPat      241 non-null    object 
 10  nameScores          241 non-null    float64
 11  matchNames          241 non-null    object 
 12  subJurisCode        241 non-null    object 
 13  incorporation_date  241 non-null    object 
 14  address_city        73 non-null     object 
 15  address_state       72 non-null     object 
 16  address_

None

,assignee_id,organization,totalScore,ID,location_id,city,state,city_latitude,city_longitude,dateOfFirstPat,...,agent_latitude,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff
0,00059b74-f414-43cb-a36e-e30fb41c9285,Qbe Holdings,24,49989,fe67d7f9-cb8f-11eb-9615-121df0c29c1e,New York,Ny,40.6931,-73.8784,2013-03-19,...,NaN,NaN,New York,Ny,40.6931,-73.8784,0.0,NaN,0.0,34.06
1,004053ef-9a48-4d23-9fd9-93ff47b7c299,Cora Aero,5,256004,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Mountain View,Ca,37.4139,-122.0850,2019-11-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.53
2,0116426f-e05d-4c31-bed1-5e811c29400a,Dohrmann Industries,15,49469,f8e66f35-cb90-11eb-9615-121df0c29c1e,Owens Crossroads,Al,33.3084,-88.0598,2000-05-30,...,NaN,NaN,NaN,Al,NaN,NaN,NaN,NaN,NaN,0.28
3,01756cf3-dcd4-4332-a3ff-a240e090c08f,Ls Biopath,24,49262,9bf15fbd-cb90-11eb-9615-121df0c29c1e,Saratoga,Ca,37.2664,-122.0260,2013-04-09,...,37.3094,-122.061,NaN,Ca,NaN,NaN,0.0,3.5,NaN,5.41
4,01756cf3-dcd4-4332-a3ff-a240e090c08f,Ls Biopath,24,49263,9eecf551-cb8f-11eb-9615-121df0c29c1e,Cupertino,Ca,37.3094,-122.0610,2013-04-09,...,37.3094,-122.061,Cupertino,Ca,37.3094,-122.0610,3.5,0.0,0.0,5.41


In [6]:
OCScoringDF1Merge1=OCScoringDF1Merge.iloc[:,[3,0,4,1,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                18,19,20,21,22,23,24,25,26,27,28,29,2]].sort_values(by=['ID'])

display(OCScoringDF1Merge1.info(),OCScoringDF1Merge1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 234 to 58
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  241 non-null    int64  
 1   assignee_id         241 non-null    object 
 2   location_id         241 non-null    object 
 3   organization        241 non-null    object 
 4   city                241 non-null    object 
 5   state               241 non-null    object 
 6   city_latitude       238 non-null    float64
 7   city_longitude      238 non-null    float64
 8   dateOfFirstPat      241 non-null    object 
 9   nameScores          241 non-null    float64
 10  matchNames          241 non-null    object 
 11  subJurisCode        241 non-null    object 
 12  incorporation_date  241 non-null    object 
 13  address_city        73 non-null     object 
 14  address_state       72 non-null     object 
 15  address_latitude    66 non-null     float64
 16  address

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,agent_longitude,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore
234,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,-79.9602,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11,25
231,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78,7
225,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41,5
219,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56,5
214,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70,7


In [7]:
OCScoringDF1Merge1['assignee_id'].nunique()

212

In [8]:
OCScoringDF1Merge1.loc[(OCScoringDF1Merge1['state']==OCScoringDF1Merge1['subJurisCode'])]['assignee_id'].nunique()

109

In [9]:
OCScoringDF1Merge1.loc[(OCScoringDF1Merge1['state']!='De') & 
                       (OCScoringDF1Merge1['subJurisCode']=='De')]['assignee_id'].nunique()

103

In [10]:
OCScoringDF1Merge1['confidenceScore']=((10-1)*((OCScoringDF1Merge1['totalScore']-min(OCScoringDF1Merge1['totalScore']))/
                                               (max(OCScoringDF1Merge1['totalScore'])-min(OCScoringDF1Merge1['totalScore']))))+1

OCScoringDF1Merge1['confidenceScore']=[round(num1, 2) for num1 in OCScoringDF1Merge1['confidenceScore']]

display(OCScoringDF1Merge1.info(),OCScoringDF1Merge1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 234 to 58
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  241 non-null    int64  
 1   assignee_id         241 non-null    object 
 2   location_id         241 non-null    object 
 3   organization        241 non-null    object 
 4   city                241 non-null    object 
 5   state               241 non-null    object 
 6   city_latitude       238 non-null    float64
 7   city_longitude      238 non-null    float64
 8   dateOfFirstPat      241 non-null    object 
 9   nameScores          241 non-null    float64
 10  matchNames          241 non-null    object 
 11  subJurisCode        241 non-null    object 
 12  incorporation_date  241 non-null    object 
 13  address_city        73 non-null     object 
 14  address_state       72 non-null     object 
 15  address_latitude    66 non-null     float64
 16  address

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,confidenceScore
234,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11,25,10.00
231,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78,7,2.64
225,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41,5,1.82
219,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56,5,1.82
214,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70,7,2.64


In [11]:
col         = 'confidenceScore'
conditions  = [ (OCScoringDF1Merge1[col] >= 1) & (OCScoringDF1Merge1[col] < 2),
                (OCScoringDF1Merge1[col] >= 2) & (OCScoringDF1Merge1[col] < 3), 
                (OCScoringDF1Merge1[col] >= 3) & (OCScoringDF1Merge1[col] < 4),
                (OCScoringDF1Merge1[col] >= 4) & (OCScoringDF1Merge1[col] < 5), 
                (OCScoringDF1Merge1[col] >= 5) & (OCScoringDF1Merge1[col] < 6), 
                (OCScoringDF1Merge1[col] >= 6) & (OCScoringDF1Merge1[col] < 7), 
                (OCScoringDF1Merge1[col] >= 7) & (OCScoringDF1Merge1[col] < 8), 
                (OCScoringDF1Merge1[col] >= 8) & (OCScoringDF1Merge1[col] < 9), 
                (OCScoringDF1Merge1[col] >= 9) & (OCScoringDF1Merge1[col] < 10),
                (OCScoringDF1Merge1[col] == 10) ]
choices     = [ 1,2,3,4,5,6,7,8,9,10 ]
    
OCScoringDF1Merge1["score"] = np.select(conditions, choices, default=np.nan)
OCScoringDF1Merge1.drop(columns=['confidenceScore'],inplace=True)
OCScoringDF1Merge2=OCScoringDF1Merge1.sort_values(by=['ID']).reset_index(drop=True)

display(OCScoringDF1Merge2.info(),OCScoringDF1Merge2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  241 non-null    int64  
 1   assignee_id         241 non-null    object 
 2   location_id         241 non-null    object 
 3   organization        241 non-null    object 
 4   city                241 non-null    object 
 5   state               241 non-null    object 
 6   city_latitude       238 non-null    float64
 7   city_longitude      238 non-null    float64
 8   dateOfFirstPat      241 non-null    object 
 9   nameScores          241 non-null    float64
 10  matchNames          241 non-null    object 
 11  subJurisCode        241 non-null    object 
 12  incorporation_date  241 non-null    object 
 13  address_city        73 non-null     object 
 14  address_state       72 non-null     object 
 15  address_latitude    66 non-null     float64
 16  address_

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,score
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11,25,10.0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78,7,2.0
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41,5,1.0
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56,5,1.0
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70,7,2.0


In [12]:
OCScoringDF1Merge3=OCScoringDF1Merge2.drop_duplicates(subset=['assignee_id','organization','state','subJurisCode'],
                                                      keep='first')
OCScoringDF1Merge4=OCScoringDF1Merge3.drop_duplicates(subset=['assignee_id','organization','incorporation_date'],
                                                      keep='first')

display(OCScoringDF1Merge4.info(),OCScoringDF1Merge4.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 0 to 239
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  214 non-null    int64  
 1   assignee_id         214 non-null    object 
 2   location_id         214 non-null    object 
 3   organization        214 non-null    object 
 4   city                214 non-null    object 
 5   state               214 non-null    object 
 6   city_latitude       213 non-null    float64
 7   city_longitude      213 non-null    float64
 8   dateOfFirstPat      214 non-null    object 
 9   nameScores          214 non-null    float64
 10  matchNames          214 non-null    object 
 11  subJurisCode        214 non-null    object 
 12  incorporation_date  214 non-null    object 
 13  address_city        69 non-null     object 
 14  address_state       68 non-null     object 
 15  address_latitude    63 non-null     float64
 16  address_

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,score
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,37.2738,-79.9602,2001-01-30,100.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3.11,25,10.0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,41.4543,-70.6038,2018-10-16,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.78,7,2.0
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,42.2187,-71.2026,2017-06-27,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41,5,1.0
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,38.8937,-76.9879,2016-07-26,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.56,5,1.0
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,36.2689,-98.4798,2004-04-20,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.70,7,2.0


In [13]:
OCScoringDF1Merge5=OCScoringDF1Merge4.groupby(by=['assignee_id','organization'],as_index=False)['incorporation_date'].min()

OCScoringDF1Merge6=OCScoringDF1Merge5.merge(OCScoringDF1Merge4,on=['assignee_id','organization','incorporation_date'],
                                           how='inner')

display(OCScoringDF1Merge6.info(),OCScoringDF1Merge6.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   assignee_id         212 non-null    object 
 1   organization        212 non-null    object 
 2   incorporation_date  212 non-null    object 
 3   ID                  212 non-null    int64  
 4   location_id         212 non-null    object 
 5   city                212 non-null    object 
 6   state               212 non-null    object 
 7   city_latitude       211 non-null    float64
 8   city_longitude      211 non-null    float64
 9   dateOfFirstPat      212 non-null    object 
 10  nameScores          212 non-null    float64
 11  matchNames          212 non-null    object 
 12  subJurisCode        212 non-null    object 
 13  address_city        68 non-null     object 
 14  address_state       67 non-null     object 
 15  address_latitude    62 non-null     float64
 16  address_

None

,assignee_id,organization,incorporation_date,ID,location_id,city,state,city_latitude,city_longitude,dateOfFirstPat,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,score
0,00059b74-f414-43cb-a36e-e30fb41c9285,Qbe Holdings,1979-03-06,49989,fe67d7f9-cb8f-11eb-9615-121df0c29c1e,New York,Ny,40.6931,-73.8784,2013-03-19,...,New York,Ny,40.6931,-73.8784,0.0,NaN,0.0,34.06,24,9.0
1,004053ef-9a48-4d23-9fd9-93ff47b7c299,Cora Aero,2019-04-26,256004,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Mountain View,Ca,37.4139,-122.0850,2019-11-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.53,5,1.0
2,0116426f-e05d-4c31-bed1-5e811c29400a,Dohrmann Industries,2000-02-17,49469,f8e66f35-cb90-11eb-9615-121df0c29c1e,Owens Crossroads,Al,33.3084,-88.0598,2000-05-30,...,NaN,Al,NaN,NaN,NaN,NaN,NaN,0.28,15,5.0
3,01756cf3-dcd4-4332-a3ff-a240e090c08f,Ls Biopath,2007-11-13,49262,9bf15fbd-cb90-11eb-9615-121df0c29c1e,Saratoga,Ca,37.2664,-122.0260,2013-04-09,...,NaN,Ca,NaN,NaN,0.0,3.5,NaN,5.41,24,9.0
4,01a0afe9-4372-4deb-abd5-678d884ed4bb,Sonifi Solutions,1993-10-07,49174,dc55d56a-cb90-11eb-9615-121df0c29c1e,Sioux Falls,Sd,43.5414,-96.7337,2014-10-28,...,NaN,Sd,NaN,NaN,NaN,188.4,NaN,21.07,16,6.0


In [14]:
OCScoringDF1Merge7=OCScoringDF1Merge6.iloc[:,[3,0,4,1,5,6,7,8,9,10,11,12,2,13,14,15,16,17,
                                              18,19,20,21,22,23,24,25,26,27,28,29,30]]

display(OCScoringDF1Merge7.info(),OCScoringDF1Merge7.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  212 non-null    int64  
 1   assignee_id         212 non-null    object 
 2   location_id         212 non-null    object 
 3   organization        212 non-null    object 
 4   city                212 non-null    object 
 5   state               212 non-null    object 
 6   city_latitude       211 non-null    float64
 7   city_longitude      211 non-null    float64
 8   dateOfFirstPat      212 non-null    object 
 9   nameScores          212 non-null    float64
 10  matchNames          212 non-null    object 
 11  subJurisCode        212 non-null    object 
 12  incorporation_date  212 non-null    object 
 13  address_city        68 non-null     object 
 14  address_state       67 non-null     object 
 15  address_latitude    62 non-null     float64
 16  address_

None

,ID,assignee_id,location_id,organization,city,state,city_latitude,city_longitude,dateOfFirstPat,nameScores,...,data_city,data_state,data_latitude,data_longitude,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,dateDiff,totalScore,score
0,49989,00059b74-f414-43cb-a36e-e30fb41c9285,fe67d7f9-cb8f-11eb-9615-121df0c29c1e,Qbe Holdings,New York,Ny,40.6931,-73.8784,2013-03-19,100.0,...,New York,Ny,40.6931,-73.8784,0.0,NaN,0.0,34.06,24,9.0
1,256004,004053ef-9a48-4d23-9fd9-93ff47b7c299,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Cora Aero,Mountain View,Ca,37.4139,-122.0850,2019-11-05,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.53,5,1.0
2,49469,0116426f-e05d-4c31-bed1-5e811c29400a,f8e66f35-cb90-11eb-9615-121df0c29c1e,Dohrmann Industries,Owens Crossroads,Al,33.3084,-88.0598,2000-05-30,100.0,...,NaN,Al,NaN,NaN,NaN,NaN,NaN,0.28,15,5.0
3,49262,01756cf3-dcd4-4332-a3ff-a240e090c08f,9bf15fbd-cb90-11eb-9615-121df0c29c1e,Ls Biopath,Saratoga,Ca,37.2664,-122.0260,2013-04-09,100.0,...,NaN,Ca,NaN,NaN,0.0,3.5,NaN,5.41,24,9.0
4,49174,01a0afe9-4372-4deb-abd5-678d884ed4bb,dc55d56a-cb90-11eb-9615-121df0c29c1e,Sonifi Solutions,Sioux Falls,Sd,43.5414,-96.7337,2014-10-28,100.0,...,NaN,Sd,NaN,NaN,NaN,188.4,NaN,21.07,16,6.0


In [15]:
finalScoresWoStates=OCScoringDF1Merge7.iloc[:,[0,1,2,3,4,5,8,9,10,11,12,13,14,17,18,21,22,29,30]].sort_values(by=['ID'])

display(finalScoresWoStates.info(),finalScoresWoStates.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 207 to 88
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  212 non-null    int64  
 1   assignee_id         212 non-null    object 
 2   location_id         212 non-null    object 
 3   organization        212 non-null    object 
 4   city                212 non-null    object 
 5   state               212 non-null    object 
 6   dateOfFirstPat      212 non-null    object 
 7   nameScores          212 non-null    float64
 8   matchNames          212 non-null    object 
 9   subJurisCode        212 non-null    object 
 10  incorporation_date  212 non-null    object 
 11  address_city        68 non-null     object 
 12  address_state       67 non-null     object 
 13  agent_city          19 non-null     object 
 14  agent_state         18 non-null     object 
 15  data_city           42 non-null     object 
 16  data_st

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state,totalScore,score
207,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,2001-01-30,100.0,The Egg Factory,Va,1997-12-22,Roanoke,Va,Roanoke,Va,NaN,NaN,25,10.0
204,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,2018-10-16,100.0,Tank Vision,De,2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,7,2.0
199,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,2017-06-27,100.0,Infinibox,De,2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN,5,1.0
194,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,2016-07-26,100.0,Fanamana,De,2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,5,1.0
189,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,2004-04-20,100.0,Pendpac,Ok,2003-08-07,NaN,NaN,NaN,NaN,NaN,NaN,7,2.0


In [16]:
OCScoringDF1Merge7['assignee_id'].nunique()

212

In [17]:
OCScoringDF1Merge7.loc[(OCScoringDF1Merge7['state']==OCScoringDF1Merge7['subJurisCode'])]['assignee_id'].nunique()

109

In [18]:
OCScoringDF1Merge7.loc[(OCScoringDF1Merge7['state']!='De') & 
                       (OCScoringDF1Merge7['subJurisCode']=='De')]['assignee_id'].nunique()

103

In [19]:
# finalScoresWoStates.to_csv("../csvResults/reviewrecords.csv",index=False)

In [20]:
### save the new dataset as a checkpoint
res_folder = "../csvResults/"
outpt_file = "OcResults1000ScoredData.csv"
a_full = os.path.join(res_folder,outpt_file)

# finalScoresWoStates.to_csv(a_full,index=False)